# Test: graphfw.core.auth.TokenProvider

Dieses Notebook testet die Authentifizierung gegen Microsoft Graph via **Client Credentials**.

## Voraussetzungen
- `pip install msal`
- Gültige App-Registrierung (Application Permissions) in Azure AD
- Konfiguration via `config.json` **oder** Umgebungsvariablen:
  - `GRAPH_TENANT_ID`, `GRAPH_CLIENT_ID`, `GRAPH_CLIENT_SECRET`

⚠️ Dieses Notebook druckt **keine** Secrets; Tokens werden maskiert angezeigt.

In [22]:
# Imports & Setup
from pathlib import Path
import pandas as pd
import sys

import sys
sys.path.insert(0, "..")  # eine Ebene hoch zum Repo-Root

from graphfw.core.auth import TokenProvider

TokenProvider.__version__

AttributeError: type object 'TokenProvider' has no attribute '__version__'

## Möglichkeit 1: TENANT_ID, CLIENT_ID, CLIENT_SECRET aus einer JSON Datei

In [19]:
# 1) Aus JSON
tp = TokenProvider.from_json(r"C:\python\Scripts\config.json")
token = tp.get_access_token()  # verwendet Graph .default
print("OK, Token-Länge:", len(token))

OK, Token-Länge: 1863


## Möglichkeit 2:  TENANT_ID, CLIENT_ID, CLIENT_SECRET aus den Umgebungsvariablen

In [20]:
tp_env = TokenProvider.from_values(
    tenant_id=os.environ["GRAPH_TENANT_ID"],
    client_id=os.environ["GRAPH_CLIENT_ID"],
    client_secret=os.environ["GRAPH_CLIENT_SECRET"],
)
token2 = tp_env.get_access_token("https://graph.microsoft.com/.default")

KeyError: 'GRAPH_TENANT_ID'

In [21]:
# 2) Aus ENV (GRAPH_TENANT_ID / GRAPH_CLIENT_ID / GRAPH_CLIENT_SECRET)
tp_env = TokenProvider.from_env()
token2, ok, err = tp_env.get_token(["https://graph.microsoft.com/.default"], return_status=True)
print("OK:", ok, "ERR:", err)

AttributeError: type object 'TokenProvider' has no attribute 'from_env'

## Möglichkeit 3: Zugangsdaten Hardcoded

In [ ]:
# 3) Direkt aus Werten (alias früher: from_client_credentials)
tp_val = TokenProvider.from_client_credentials(
    tenant_id="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
    client_id="aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee",
    client_secret="***"
)
token3 = tp_val.get_access_token(force_refresh=False)

In [ ]:
from pathlib import Path
import os, time
from graphfw.core.auth import TokenProvider

# Wähle einen Modus: 'json' oder 'env'
MODE = os.getenv("AUTH_TEST_MODE", "env")  # 'json' | 'env'
CONFIG_PATH = os.getenv("AUTH_CONFIG_PATH", "config.json")

if MODE == "json" and not Path(CONFIG_PATH).exists():
    raise FileNotFoundError(f"Config-Datei nicht gefunden: {CONFIG_PATH}")

if MODE == "json":
    tp = TokenProvider.from_json(CONFIG_PATH)
else:
    # Erwartet: GRAPH_TENANT_ID / GRAPH_CLIENT_ID / GRAPH_CLIENT_SECRET
    tp = TokenProvider.from_env()

tp

In [ ]:
SCOPE = "https://graph.microsoft.com/.default"

t0 = time.time()
token = tp.get_access_token(SCOPE)
t1 = time.time()

masked = token[:16] + "…" + token[-8:]
print("Token erhalten (maskiert):", masked)
print("Dauer (s):", round(t1 - t0, 3))

## Optional: Force-Refresh
Erzwingt das Umgehen des Caches.

In [ ]:
t0 = time.time()
token2 = tp.get_access_token(SCOPE, force_refresh=True)
t1 = time.time()
print("Token (neu, maskiert):", token2[:16] + "…" + token2[-8:])
print("Dauer (s):", round(t1 - t0, 3))
print("Gleiches Token wie zuvor?", token2 == token)